In [ ]:
import music21
from tqdm import tqdm
import numpy as np

In [ ]:
import json

with open('../data/midi_frequencies.json') as file:
    midi_freq = json.load(file)
    
with open('../data/rhythm_frequencies.json') as file:
    rhythm_freq = json.load(file)

In [ ]:
def convert_rhythm_freq_to_prob(rhythm_freq):
    """
    Convert rhythm frequencies to probabilities. The returned rhythm_prob will be a
    dict whose keys are sub-beats, and whose values are lists of 2 lists. The first
    list contains the durations (rhythms), and the second list contains the corresponding
    probabilities. Two lists are necessary in order to later use np.random.choice().
    """
    rhythm_prob = {}
    for beat, durations in rhythm_freq.items():
        duration_list = []
        freq_list = []
        
        for duration, freq in durations.items():
            duration_list.append(float(duration))
            freq_list.append(freq)
            
        prob_list = (freq_list / np.sum(freq_list)).tolist()
        rhythm_prob[float(beat)] = [duration_list, prob_list]
        
    return rhythm_prob

In [ ]:
def convert_midi_freq_to_prob(midi_freq, lowest_midi, highest_midi):    
    """
    Convert MIDI frequencies to probabilities. The returned midi_prob will be a
    dict whose keys are the previous MIDI note, and whose values are lists of 2 lists.
    The first list contains the next MIDI notes, and the second list contains the corresponding
    probabilities. Two lists are necessary in order to later use np.random.choice().
    """
    midi_prob = {}
    for prev_midi in range(lowest_midi, highest_midi + 1):
        midi_list = []
        freq_list = []
        
        for next_midi in range(lowest_midi, highest_midi + 1):
            midi_list.append(next_midi)
            freq_list.append(midi_freq[prev_midi][next_midi])
        
        prob_list = (freq_list / np.sum(freq_list)).tolist()
        midi_prob[prev_midi] = [midi_list, prob_list]
    
    return midi_prob

In [ ]:
# Range of notes the final generated piece will contain
violin_lowest_pitch = music21.pitch.Pitch('G3')
violin_highest_pitch = music21.pitch.Pitch('D6')

def get_midi_prob_for_key(midi_freq, key):
    """
    Gets the MIDI probability dict for a given key, to be used in the generate_music() function.
    This function sets the lowest and highest pitches so that once the piece is transposed to the
    given key, the transposed lowest and highest notes will be in accordance with the violin range.
    """
    tp_interval = music21.interval.Interval(music21.pitch.Pitch('C5'),
                                                   music21.pitch.Pitch(key.tonic, octave=5)).reverse()
    lowest_pitch = violin_lowest_pitch.transpose(tp_interval)
    highest_pitch = violin_highest_pitch.transpose(tp_interval)
    return convert_midi_freq_to_prob(midi_freq, lowest_pitch.midi, highest_pitch.midi)

In [ ]:
def generate_music(midi_prob, rhythm_prob, piece_duration, key=music21.key.Key('C'), title="Composition"):
    """
    Generate a composition using the MIDI and rhythm probabilities given, and in the key given. The piece
    will be in the range for a violin to play.
    """
    piece = music21.stream.Stream()
    piece.insert(0, music21.key.Key('C'))
    piece.insert(0, music21.meter.TimeSignature('4/4'))
    
    piece.insert(0, music21.metadata.Metadata())
    piece.metadata.title = title
    piece.metadata.composer = "Chenchen Gu (probabilistically generated)"
    
    prev_note = music21.note.Note('C5', quarterLength=1)
    piece.append(prev_note)
    cur_beat = prev_note.quarterLength
    
    while cur_beat < piece_duration:
        # Get pitch
        prev_midi = prev_note.pitch.midi
        cur_midi = np.random.choice(midi_prob[prev_midi][0], p=midi_prob[prev_midi][1])
        cur_note = music21.note.Note(midi=cur_midi)
        piece.append(cur_note)
        
        # Get duration/rhythm
        cur_sub_beat = cur_beat % 1.0
        cur_duration = np.random.choice(rhythm_prob[cur_sub_beat][0], p=rhythm_prob[cur_sub_beat][1])
        cur_note.quarterLength = cur_duration
        cur_beat += cur_duration
    
    # Transpose piece to given key
    transpose_interval = music21.interval.Interval(music21.pitch.Pitch('C5'),
                                                   music21.pitch.Pitch(key.tonic, octave=5))
    piece.transpose(transpose_interval, inPlace=True)
    
    # Simplify notes to enharmonics in key signature to reduce accidentals
    for note in piece.recurse().notes:
        note.pitches = music21.pitch.simplifyMultipleEnharmonics(note.pitches, keyContext=key)
    
    return piece

In [ ]:
# Rhythm probabilities only need to be computed once
rhythm_prob = convert_rhythm_freq_to_prob(rhythm_freq)

In [ ]:
# MIDI pitch probabilities need to be recomputed every time a new key is used
piece_key = music21.key.Key('E')
midi_prob = get_midi_prob_for_key(midi_freq, piece_key)

In [ ]:
piece = generate_music(midi_prob, rhythm_prob, 60, piece_key, "Example composition")
piece.show()

In [ ]:
keys = ['A-', 'A', 'B-', 'C', 'D', 'E-', 'E', 'F', 'G']
piece_len = 60

for i in tqdm(range(1, 21)):
    cur_key = music21.key.Key(np.random.choice(keys))
    piece = generate_music(midi_prob, rhythm_prob, piece_len, cur_key, f"Composition {i}")
    piece.write('musicxml', fp=f'../generated_music/composition_{i:02}.xml')
    piece.write('musicxml.pdf', fp=f'../generated_music/composition_{i:02}.pdf')